## Исследование надёжности заёмщиков - анализ банковских данных

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Загрузка файла с данными и изучение общей информации

In [2]:
#получим общую информацию о таблице
print('Общая информация о таблице:')
print(df.info())

Общая информация о таблице:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


In [3]:
#получим первые пять строк таблицы df
print('Первые пять строк таблицы df:')
df.head()

Первые пять строк таблицы df:


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

Всего в таблице 12 столбцов:

    children — количество детей в семье
    days_employed — общий трудовой стаж в днях
    dob_years — возраст клиента в годах
    education — уровень образования клиента
    education_id — идентификатор уровня образования
    family_status — семейное положение
    family_status_id — идентификатор семейного положения
    gender — пол клиента
    income_type — тип занятости
    debt — имел ли задолженность по возврату кредитов
    total_income — ежемесячный доход
    purpose — цель получения кредита


Типы данных столбцов различаются: object, float64, int64.
Количество значений в столбцах также различается: в столбцах 'days_employed' и 'total_income' указанные значения меньше, чем в остальных. Значит, в данных есть пропущенные значения.


### Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
#получим суммарное количество пропусков для каждого столбца, для этого применим метод isnull()
print('Cуммарное количество пропусков для каждого столбца до обработки пропусков:')
print(df.isnull().sum())

Cуммарное количество пропусков для каждого столбца до обработки пропусков:
children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


In [5]:
#проверим, значения в столбцах 'total_income', days_employed' пропущены в одних и тех же строках или нет
print('Cуммарное количество пропусков при условии, что значения пропущены и в total_income, и в days_employed:')
print(df[(df['total_income'].isnull()) & (df['days_employed'].isnull())].isnull().sum())

Cуммарное количество пропусков при условии, что значения пропущены и в total_income, и в days_employed:
children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


#### Вывод

В столбцах 'days_employed', 'total_income' по 2174 пропущенных значений. Значения пропущены в одних и тех же строках.

#### Столбец 'children'

In [6]:
#выделим уникальные значения в столбце children
print('Уникальные значения в столбце children:')
print(df['children'].value_counts())

Уникальные значения в столбце children:
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


In [7]:
#скореектируем подозрительные значения
df.loc[(df.children == 20), 'children'] = 2
df.loc[(df.children == -1), 'children'] = 1

In [8]:
#проверим уникальные значения в столбце children после корректировки
print('Уникальные значения в столбце children после корректировки:')
print(df['children'].value_counts())

Уникальные значения в столбце children после корректировки:
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


#### Вывод

При рассмотрении уникальных значений в столбце 'children' встречаются подозрительные значения -1 и 20. Считаем, что это опечатка при внесении данных. Скорректируем значение 20 на 2, -1 на 1. После правок значения выглядят корректно. Наибольшее количество заёмщиков в выгрузке не имеет детей, такая категория граждан смелее берёт на себя кредитные обязательства.

#### Столбец 'days_employed'

In [9]:
#посмотрим, какие значения (помимо пропущенных) встречаются в столбце days_employed
print(df['days_employed'].describe())

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64


In [10]:
#избавимся от минусов в столбце days_employed
df['days_employed'] = df['days_employed'].abs()

In [11]:
#посмотрим, какие значения встречаются в столбце days_employed после избавления от минусов
print(df['days_employed'].describe())

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64


In [12]:
#получим первые пять строк таблицы df после удаления минусов в столбце days_employed
print('Первые пять строк таблицы df после удаления минусов в столбце days_employed:')
df.head()

Первые пять строк таблицы df после удаления минусов в столбце days_employed:


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


#### Вывод

В столбце 'days_employed' присутствуют не только 2174 пропущенных значения, но и отрицательные значения, что некорректно для обозначения общего трудового стажа в днях.
Избавляемся от минусов при помощи метода abs().

In [13]:
#посмотрим, какой максимальный стаж в годах
max_days_employed = df['days_employed'].max() / 365
print('Максимальный трудовой стаж в наборе данных:', max_days_employed, 'лет')

Максимальный трудовой стаж в наборе данных: 1100.6997273296713 лет


In [14]:
#в итернете возьмём среднее значение трудового стажа в годах 
work_experience_years = 35
print('Средний трудовой стаж:', work_experience_years, 'лет')
print()
#переведём средний трудовой стаж в дни
work_experience_days = work_experience_years * 365
print('Средний трудовой стаж:', work_experience_days, 'дней')

Средний трудовой стаж: 35 лет

Средний трудовой стаж: 12775 дней


In [15]:
#напишем функцию, которая сравнивает значение в столбце со средним значением и при превышении делит на 24
def days_employed(row):
    if row > work_experience_days:
        return row / 24
    else:
        return row
        
#обновим значения в столбце
df['days_employed'] = df['days_employed'].apply(days_employed)

In [16]:
#посмотрим, какие значения встречаются в столбце days_employed после корректировки
print(df['days_employed'].describe())

count    19351.000000
mean      4599.481030
std       5333.703341
min         24.141633
25%        916.345592
50%       2168.517932
75%       5453.955842
max      16739.808353
Name: days_employed, dtype: float64


In [17]:
#посмотрим, какой максимальный стаж в годах после корректировки
max_days_employed_1 = df['days_employed'].max() / 365
print('Максимальный трудовой стаж в годах после корректировки:', max_days_employed_1, 'лет')

Максимальный трудовой стаж в годах после корректировки: 45.8624886387363 лет


In [18]:
#получим первые пять строк таблицы df после корректировки стажа в столбце days_employed
print('Первые пять строк таблицы df после корректировки стажа в столбце days_employed:')
df.head()

Первые пять строк таблицы df после корректировки стажа в столбце days_employed:


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


#### Вывод

Также в столце 'days_employed' для некоторых позиций указан трудовой стаж, невозможный для среднестатистического землянина (401755 дней - это более 1000 лет трудового стажа). Возможно, стаж ошибочно записывали не в днях, а в часах.
Тогда переведём такие значения в дни. Отталкиваться будем от значения среднего трудового стажа в 35 лет или 12775 дней (сведения из интернета). Все значения, которые превышают средний трудовой стаж, поделим на 24. Напишем функцию, которая сравнивает значение в столбце со средним значением и при превышении делит на 24.
Максимальный трудовой стаж в наборе данных после корректировки: 45 лет.
Минимальный трудовой стаж в наборе данных: 24 дня.

In [19]:
#сгруппируем по типу занятости и посмотрим количество в каждой группе и медианное значение трудового стажа для каждой группы
print(df.groupby('income_type')['days_employed'].agg(['count', 'median']))

                 count        median
income_type                         
безработный          2  15267.235531
в декрете            1   3296.759962
госслужащий       1312   2640.576698
компаньон         4577   1526.115535
пенсионер         3443  15217.221094
предприниматель      1    520.848083
сотрудник        10014   1563.230174
студент              1    578.751554


In [20]:
#создадим словарь, где ключи - это значения столбца income_type, а значения по ключам - соответствующие медианы
days_employed_dict = df.groupby('income_type')['days_employed'].median().to_dict()

#напишем функцию, которая заполняет пропущенные значения трудового стажа соответствующим данной группе значением из словаря
def fill_days_employed(row):
    #проверяем значение в столбце days_employed является пропущенным или нет
    if pd.isna(row['days_employed']):
        #если значение пропущено, то берем медиану для заполнения из словаря
        return days_employed_dict[row['income_type']]
    #если значение не пропущено, то возвращаем его
    else:
        return row['days_employed']

df['days_employed'] = df.apply(fill_days_employed, axis=1)

In [21]:
#убедимся, что после замены пропущенных значений столбец days_employed не содержит пропусков
print('Cуммарное количество пропусков для столбца days_employed после обработки пропусков:')
print(df['days_employed'].isnull().sum())

Cуммарное количество пропусков для столбца days_employed после обработки пропусков:
0


#### Вывод

Для замены пропущенных значений в столбце 'days_employed' сгруппируем значения по типу занятости и посмотрим количество в каждой группе и медианное значение трудового стажа для каждой группы. Далее создадим из сгруппированных значений словарь,  где ключи - это значения столбца 'income_type', а значения по ключам - соответствующие медианы, и напишем функцию, которая проверяет, отсутствует ли значение в столбце 'days_employed', и если отсутствует, то заменяет медианным значением для данной группы.
После замены пропущенных значений убедимся, что суммарное количество пропусков для данного столбца равно нулю.

#### Столбец 'dob_years'

In [22]:
#выделим уникальные значения в столбце dob_years
print('Уникальные значения в столбце dob_years:')
print(df['dob_years'].value_counts())

Уникальные значения в столбце dob_years:
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


In [23]:
#выявим самого взрослого заёмщика
df['dob_years'].max()

75

In [24]:
#выявим самого юного заёмщика
df['dob_years'].min()

0

In [25]:
#найдём медианное значение для данного столбца
df['dob_years'].median()

42.0

In [26]:
#скореектируем подозрительные значения
df.loc[(df.dob_years == 0), 'dob_years'] = 42

In [27]:
#выделим уникальные значения в столбце dob_years после корректировки
print('Уникальные значения в столбце dob_years после корректировки:')
print(df['dob_years'].value_counts())

Уникальные значения в столбце dob_years после корректировки:
42    698
35    617
40    609
41    607
34    603
38    598
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


In [28]:
#выявим самого юного заёмщика после корректировки
df['dob_years'].min()

19

#### Вывод

В столбце dob_years присутствуют совсем юные заёмщики, возраст которых - нуль. Заменим данное значение медианным для столбца. После корректировки данные отображаются корректно.
Самый юный заёмщик возрастом 19 лет.
Самый взрослый заёмщик возрастом 75 лет.

#### Столбец 'education'

In [29]:
#выделим уникальные значения в столбце education
print('Уникальные значения в столбце education:')
print(df['education'].value_counts())

Уникальные значения в столбце education:
среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64


#### Вывод

В столбце education данные отображаются корректно, регистр скорректируем на шаге "Обработка дубликатов"

#### Столбец 'education_id'

In [30]:
#выделим уникальные значения в столбце education_id
print('Уникальные значения в столбце education_id:')
print(df['education_id'].value_counts())

Уникальные значения в столбце education_id:
1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64


#### Вывод

В столбце education_id данные отображаются корректно, правки вносить не требуется

#### Столбец 'family_status'

In [31]:
#выделим уникальные значения в столбце family_status
print('Уникальные значения в столбце family_status:')
print(df['family_status'].value_counts())

Уникальные значения в столбце family_status:
женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


#### Вывод

В столбце family_status данные отображаются корректно, правки вносить не требуется

#### Столбец 'family_status_id'

In [32]:
#выделим уникальные значения в столбце family_status_id
print('Уникальные значения в столбце family_status_id:')
print(df['family_status_id'].value_counts())

Уникальные значения в столбце family_status_id:
0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64


#### Вывод

В столбце family_status_id данные отображаются корректно, правки вносить не требуется

#### Столбец 'gender'

In [33]:
#выделим уникальные значения в столбце gender
print('Уникальные значения в столбце gender:')
print(df['gender'].value_counts())

Уникальные значения в столбце gender:
F      14236
M       7288
XNA        1
Name: gender, dtype: int64


#### Вывод

В столбцe 'gender' встречается пол, расшифровку которого найти не удалось. Если этот пол - некий существующий формат, то значит выгрузка получена не от российского банка. Удалять данное значение не будем, т.к. на поставленные задачи это не влияет.

Среди заёмщиков больше представителей женского пола  - 14236 человек.

#### Столбец 'income_type'

In [34]:
#выделим уникальные значения в столбце с типом занятости income_type
print('Тип занятости:')
print(df['income_type'].value_counts())

Тип занятости:
сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64


#### Вывод

В столбце income_type данные отображаются корректно, правки вносить не требуется

#### Столбец 'debt'

In [35]:
#выделим уникальные значения в столбце debt
print('Уникальные значения в столбце debt:')
print(df['debt'].value_counts())

Уникальные значения в столбце debt:
0    19784
1     1741
Name: debt, dtype: int64


In [36]:
#посмотрим средний процент просрочки
print(df['debt'].mean()*100)

8.088269454123113


#### Вывод

В столбце debt данные отображаются корректно, правки вносить не требуется.
Средний процент просрочки составляет 8%.

#### Столбец 'total_income'

In [37]:
#посмотрим, какие значения встречаются в столбце total_income
print(df['total_income'].describe())

count    1.935100e+04
mean     1.674223e+05
std      1.029716e+05
min      2.066726e+04
25%      1.030532e+05
50%      1.450179e+05
75%      2.034351e+05
max      2.265604e+06
Name: total_income, dtype: float64


In [38]:
#сгруппируем по типу занятости и посмотрим количество в каждой группе и медианное значение дохода для каждой группы
print(df.groupby('income_type')['total_income'].agg(['count', 'median']))

                 count         median
income_type                          
безработный          2  131339.751676
в декрете            1   53829.130729
госслужащий       1312  150447.935283
компаньон         4577  172357.950966
пенсионер         3443  118514.486412
предприниматель      1  499163.144947
сотрудник        10014  142594.396847
студент              1   98201.625314


In [39]:
#создадим словарь, где ключи - это значения столбца income_type, а значения по ключам - соответствующие медианы
total_income_dict = df.groupby('income_type')['total_income'].median().to_dict()

#напишем функцию, которая заполняет пропущенные значения трудового стажа соответствующим данной группе значением из словаря
def fill_total_income(row):
    #проверяем значение в столбце days_employed является пропущенным или нет
    if pd.isna(row['total_income']):
        #если значение пропущено, то берем медиану для заполнения из словаря
        return total_income_dict[row['income_type']]
    #если значение не пропущено, то возвращаем его
    else:
        return row['total_income']

df['total_income'] = df.apply(fill_total_income, axis=1)

In [40]:
#максимальное значение дохода заёмщика
print('Максимальное значение дохода заёмщика', df['total_income'].max())

Максимальное значение дохода заёмщика 2265604.028722744


In [41]:
#минимальное значение дохода заёмщика
print('Минимальное значение дохода заёмщика', df['total_income'].min())

Минимальное значение дохода заёмщика 20667.26379327158


In [42]:
#убедимся, что после замены пропущенных значений столбец total_income не содержит пропусков
print('Cуммарное количество пропусков для столбца total_income после обработки пропусков:')
print(df['total_income'].isnull().sum())

Cуммарное количество пропусков для столбца total_income после обработки пропусков:
0


#### Вывод

Для замены пропущенных значений в столбце 'total_income' сгруппируем значения по типу занятости и посмотрим количество в каждой группе и медианное значение дохода для каждой группы. Далее создадим из сгруппированных значений словарь,  где ключи - это значения столбца 'income_type', а значения по ключам - соответствующие медианы, и напишем функцию, которая проверяет, отсутствует ли значение в столбце 'total_income', и если отсутствует, то заменяет медианным значением для данной группы.
После замены пропущенных значений убедимся, что суммарное количество пропусков для данного столбца равно нулю.

Максимальное значение дохода заёмщика в наборе данных: 2265604.028722744
Минимальное значение дохода заёмщика в наборе данных: 20667.26379327158

#### Столбец 'purpose'

In [43]:
print('Уникальные значения в столбце purpose:')
print(df['purpose'].value_counts())

Уникальные значения в столбце purpose:
свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля             

#### Вывод

В столбце purpose данные отображаются корректно, правки вносить не требуется

In [44]:
#убедимся, что после замены таблица больше не содержит пропусков
print('Cуммарное количество пропусков для каждого столбца после обработки пропусков:')
print(df.isnull().sum())

Cуммарное количество пропусков для каждого столбца после обработки пропусков:
children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


### Вывод

Для набора данных изучили данные в каждом столбце, обработали пропущенные значения, выявили артефакты, скорректировали подозрительные значения. 
После произведенных замен таблица больше не содержит пропусков.

### Замена типа данных

In [45]:
#в столбцах 'days_employed', 'total_income' заменим вещественный тип данных на целочисленный
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

In [46]:
#получим общую информацию о таблице после заполнения пропущенных значений и замены типа данных
print('Общая информация о наборе данных после заполнения пропущенных значений и замены типа данных:')
print(df.info())

Общая информация о наборе данных после заполнения пропущенных значений и замены типа данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
None


### Вывод

Для замены вещественного типа float64 на целочисленный int64 используем метод astype() с аргументом ('int')
После проведения замены в столбцах 'days_employed', 'total_income' тип данных указан int64

### Обработка дубликатов

In [47]:
#посчитаем количество дубликатов в наборе данных
print('Количество дубликатов в наборе данных до обработки:')
print(df.duplicated().sum()) 

Количество дубликатов в наборе данных до обработки:
55


In [48]:
#выведем строки с дубликатами
print('Строки с дубликатами:')
df[df.duplicated()].head()

Строки с дубликатами:


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,1563,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи
4182,1,1563,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,142594,свадьба
4851,0,15217,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба
5557,0,15217,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
7808,0,15217,57,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы


In [49]:
#переведём значения в столбце education в нижний регистр
df['education'] = df['education'].str.lower()

In [50]:
#посчитаем количество дубликатов в наборе данных после перевода значений в столбце education в нижний регистр
print('Количество дубликатов в наборе данных после перевода значений в столбце education в нижний регистр:')
print(df.duplicated().sum()) 

Количество дубликатов в наборе данных после перевода значений в столбце education в нижний регистр:
72


In [51]:
#удалим дубликаты
df = df.drop_duplicates().reset_index()

In [52]:
#проверим, оставлись ли дубликаты
print('Количество дубликатов в наборе данных после обработки:')
print(df.duplicated().sum()) 

Количество дубликатов в наборе данных после обработки:
0


### Вывод

Поиск дубликатов (грубых повторов) осуществляем при помощи метода duplicated(). Количество дубликатов посчитаем при помощи метода sum(). В наборе данных оказалось 54 строк с дубликатами.
В столбце 'education' часть значений отличается регистром. Переведём значения в данном столбце в нижний регистр методом str.lower(). После перевода значений  в нижний регистр и подсчёта количества дубликатов значение дубликатов оказалось равным 72.
Для общего количества строк (21525) это небольшое значение. Возможно, дубликаты появились из-за технологической ошибки.
После удаления количество повторов равно нулю.

### Лемматизация

In [53]:
#импортируем библиотеку pymystem3 для выделения леммы в значениях столбца с целями получения кредита
from pymystem3 import Mystem
m = Mystem()

In [54]:
#лемматизируем слова в столбце с целями получения кредита
def lemma(row):
    #результат лемматизации - список слов - склеим при помощи метода join()
    return ''.join(m.lemmatize(row['purpose']))

In [55]:
#добавим столбец с лемматизированными словами к набору данных
df['lemmas'] = df.apply(lemma, axis = 1)

In [56]:
#получим первые пять строк таблицы df с добавленным столбцом lemmas 
print('Первые пять строк таблицы data с добавленным столбцом lemmas:')
df.head()

Первые пять строк таблицы data с добавленным столбцом lemmas:


,index,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas
0,0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье\n
1,1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль\n
2,2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье\n
3,3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование\n
4,4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба\n


### Вывод

Для проведения лемматизации целей кредита используем библиотеку pymystem3. Леммы в столбце 'purpose' выделяем при помощи метода m.lemmatize (). Лемматизированнные слова склеим вызовом метода join(), выделим в отдельный столбец и добавим к общему набору данных.


### Категоризация данных

#### Категоризация данных о целях кредитов

In [57]:
#категоризируем данные о целях кредитов
#выделим уникальные цели кредитов
print('Уникальные цели кредитов:')
print(df['lemmas'].value_counts())

Уникальные цели кредитов:
автомобиль\n                                972
свадьба\n                                   791
на проведение свадьба\n                     767
сыграть свадьба\n                           765
операция с недвижимость\n                   675
покупка коммерческий недвижимость\n         661
операция с жилье\n                          652
покупка жилье для сдача\n                   651
операция с коммерческий недвижимость\n      650
жилье\n                                     646
покупка жилье\n                             646
покупка жилье для семья\n                   638
строительство собственный недвижимость\n    635
недвижимость\n                              633
операция со свой недвижимость\n             627
строительство жилой недвижимость\n          624
покупка недвижимость\n                      621
покупка свой жилье\n                        620
строительство недвижимость\n                619
ремонт жилье\n                              607
покупка жилой 

In [58]:
#напишем функцию, которая берёт на вход цель кредита и возвращает одну из категорий: автомобиль, образование, недвижимость, свадьба
def purpose_category(row):
    lemmas = ' '.join(m.lemmatize(row))
    if 'авто' in lemmas:
        return 'автомобиль'
    elif 'образов' in lemmas:
        return 'образование'
    elif 'недвиж' in lemmas or 'жиль' in lemmas:
        return 'недвижимость'
    else:
        return 'свадьба'

In [59]:
#получим столбец с категориями purpose_category
df['purpose_category'] = df['lemmas'].apply(purpose_category)

#### Вывод

Для категоризации данных о целях кредитов для начала рассмотрим уникальные значения в столбце с целями кредитов. Уникальные значения выделяем при помощи метода value_counts(). Из представленных целей можно выделить несколько категорий - свадьба, недвижимость, автомобиль, образование. Будем распределять значения по данным категориям.
Для этого напишем функцию, которая берёт на вход цель кредита и возвращает категорию. При помощи метода apply() применим написанную функцию к столбцу 'lemmas'.

#### Категоризация данных о доходах

In [60]:
#поделим значения на равные группы при помощи метода qcut()
pd.qcut(df['total_income'], 6)

0          (228882.0, 2265604.0]
1        (92140.667, 119260.667]
2           (142594.0, 172357.0]
3          (228882.0, 2265604.0]
4           (142594.0, 172357.0]
                  ...           
21448       (172357.0, 228882.0]
21449       (142594.0, 172357.0]
21450     (20666.999, 92140.667]
21451      (228882.0, 2265604.0]
21452     (20666.999, 92140.667]
Name: total_income, Length: 21453, dtype: category
Categories (6, interval[float64]): [(20666.999, 92140.667] < (92140.667, 119260.667] < (119260.667, 142594.0] < (142594.0, 172357.0] < (172357.0, 228882.0] < (228882.0, 2265604.0]]

In [61]:
#получим столбец с группами
df['total_income_group'] = pd.qcut(df['total_income'], 6)

#### Вывод

Для категоризации данных о доходах применим метод qcut() к столбцу total_income. Данный метод позволяет поделить значения на равные группы. Добавим группы в отдельный столбец total_income_group.

#### Категоризация данных о детях

In [62]:
#напишем функцию, которая берёт на вход информацию о количестве детей и возвращает категорию
def children_category(row):
    if row == 0:
        return 'нет детей'
    elif row == 1:
        return 'один ребёнок'
    elif row == 2:
        return 'двое детей'
    elif row == 3:
        return 'трое детей'
    elif row == 4:
        return 'четверо детей'
    else:
        return 'пятеро детей'

In [63]:
#получим столбец с категориями children_category
df['children_category'] = df['children'].apply(children_category)

#### Вывод

Для категоризации данных о детях напишем функцию, которая берёт на вход информацию о количестве детей и возвращает категорию. В качестве категорий расссматриваем следующие: нет детей, один ребёнок, двое детей, трое детей, четверо детей, пятеро детей. При помощи метода apply() применим написанную функцию к столбцу 'children'.

In [64]:
#получим первые пять строк таблицы df с добавленными столбцами с категориями
print('Первые пять строк таблицы df с добавленными столбцами:')
df.head()

Первые пять строк таблицы df с добавленными столбцами:


,index,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,purpose_category,total_income_group,children_category
0,0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье\n,недвижимость,"(228882.0, 2265604.0]",один ребёнок
1,1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль\n,автомобиль,"(92140.667, 119260.667]",один ребёнок
2,2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье\n,недвижимость,"(142594.0, 172357.0]",нет детей
3,3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование\n,образование,"(228882.0, 2265604.0]",трое детей
4,4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба\n,свадьба,"(142594.0, 172357.0]",нет детей


### Вывод

Провели категоризацию данных о целях кредитов, доходах, детях. После категоризации таблицу с добавленными столбцами выводим на экран.
Теперь можно группировать по созданным категориям.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [65]:
#выведем статистику по категориям наличия детей
print('Cтатистика по категориям наличия детей:')
print(df['children_category'].value_counts())

Cтатистика по категориям наличия детей:
нет детей        14090
один ребёнок      4855
двое детей        2128
трое детей         330
четверо детей       41
пятеро детей         9
Name: children_category, dtype: int64


In [66]:
#создадим сводную таблицу для обобщения данных
print('Сводная таблица:')
df_pivot_children = df.pivot_table(index = 'children_category', columns='debt', values='children', aggfunc='count')
df_pivot_children

Сводная таблица:


debt,0,1
children_category,,
двое детей,1926.0,202.0
нет детей,13027.0,1063.0
один ребёнок,4410.0,445.0
пятеро детей,9.0,NaN
трое детей,303.0,27.0
четверо детей,37.0,4.0


In [67]:
#вычислим % просроченных и возвращённых кредитов для каждой категории
df_pivot_children['% просрочки'] = df_pivot_children[1] / (df_pivot_children[0] + df_pivot_children[1])*100

df_pivot_children['% возврата'] = df_pivot_children[0] / (df_pivot_children[0] + df_pivot_children[1])*100

In [68]:
#рассмотрим таблицу с добавленными столбцами
print('Сводная таблица с добавленными столбцами:')
df_pivot_children

Сводная таблица с добавленными столбцами:


debt,0,1,% просрочки,% возврата
children_category,,,,
двое детей,1926.0,202.0,9.492481,90.507519
нет детей,13027.0,1063.0,7.544358,92.455642
один ребёнок,4410.0,445.0,9.165808,90.834192
пятеро детей,9.0,NaN,NaN,NaN
трое детей,303.0,27.0,8.181818,91.818182
четверо детей,37.0,4.0,9.756098,90.243902


### Вывод

Для вычисления % просроченных и возвращённых кредитов сначала построим сводную таблицу с количеством элементов для каждой категории. 
Сводную таблицу создаём при помощи метода pivot_table ().
Для вычисления % просроченных кредитов создаем новый столбец. Для этого поделим значения в столбце 1 (есть задолжность) на суммарное значение для категории.
Для вычисления % возвращённых кредитов также создаем новый столбец, для чего поделим значения в столбце 0 (нет задолжности) на суммарное значение для категории.

В результате видим, что для категории людей, у которых пятеро детей, данные по невозвращённым кредитам отсутствуют, т.е. формально они самые ответственные, но группа немногочисленная, поэтому не будем брать их в расчёт.

Самый низкий процент просрочки кредита, и, соответственно, самый высокий процент возврата кредитов у людей без детей.
Далее в рейтинге идут люди, у которых трое детей. Третье место по просрочке кредитов у людей с одним ребёнком. Четвертое место по просрочке кредитов у людей с двумя детьми. И завершают рейтинг люди с четырьмя детьми, у них самый высокий процент просрочки кредитов.

В шагах ниже используем аналогичные для данного шага выкладки.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [69]:
#выделим уникальные значения в столбце с семейным положением
print('Семейное положение:')
print(df['family_status'].value_counts())

Семейное положение:
женат / замужем          12339
гражданский брак          4150
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64


In [70]:
#создадим сводную таблицу для обобщения данных
print('Сводная таблица:')
df_pivot_family_status = df.pivot_table(index = 'family_status', columns='debt', values='children', aggfunc='count')
df_pivot_family_status 

Сводная таблица:


debt,0,1
family_status,,
Не женат / не замужем,2536,274
в разводе,1110,85
вдовец / вдова,896,63
гражданский брак,3762,388
женат / замужем,11408,931


In [71]:
#вычислим % просроченных и возвращённых кредитов для каждой категории
df_pivot_family_status['% просрочки'] = df_pivot_family_status[1] / (df_pivot_family_status[0] + df_pivot_family_status[1])*100

df_pivot_family_status['% возврата'] = df_pivot_family_status[0] / (df_pivot_family_status[0] + df_pivot_family_status[1])*100

In [72]:
#рассмотрим таблицу с добавленными столбцами
print('Сводная таблица с добавленными столбцами:')
df_pivot_family_status

Сводная таблица с добавленными столбцами:


debt,0,1,% просрочки,% возврата
family_status,,,,
Не женат / не замужем,2536,274,9.750890,90.249110
в разводе,1110,85,7.112971,92.887029
вдовец / вдова,896,63,6.569343,93.430657
гражданский брак,3762,388,9.349398,90.650602
женат / замужем,11408,931,7.545182,92.454818


### Вывод

После проведения вычислений видим, что самый низкий процент просрочки кредита, и, соответственно, самый высокий процент возврата кредитов у вдовствующих граждан.
Далее в рейтинге идут разведённые заёмщики. Третье место по просрочке кредитов у женатых людей. Четвертое место по просрочке кредитов у тех, кто живёт гражданским браком. И замыкают рейтинг неженатые граждане, у них самый высокий процент просрочки кредитов и, соответственно, самый низкий процент возврата.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [73]:
#выведем статистику по категориям уровней дохода
print('Cтатистика по категориям уровней дохода:')
print(df['total_income_group'].value_counts())

Cтатистика по категориям уровней дохода:
(119260.667, 142594.0]     3691
(142594.0, 172357.0]       3594
(228882.0, 2265604.0]      3576
(20666.999, 92140.667]     3576
(92140.667, 119260.667]    3575
(172357.0, 228882.0]       3441
Name: total_income_group, dtype: int64


In [74]:
#создадим сводную таблицу для обобщения данных
print('Сводная таблица:')
df_pivot_total_income = df.pivot_table(index = 'total_income_group', columns='debt', values='children', aggfunc='count')
df_pivot_total_income

Сводная таблица:


debt,0,1
total_income_group,,
"(20666.999, 92140.667]",3291,285
"(92140.667, 119260.667]",3279,296
"(119260.667, 142594.0]",3362,329
"(142594.0, 172357.0]",3291,303
"(172357.0, 228882.0]",3163,278
"(228882.0, 2265604.0]",3326,250


In [75]:
#вычислим % просроченных и возвращённых кредитов для каждой категории
df_pivot_total_income['% просрочки'] = df_pivot_total_income[1] / (df_pivot_total_income[0] + df_pivot_total_income[1])*100

df_pivot_total_income['% возврата'] = df_pivot_total_income[0] / (df_pivot_total_income[0] + df_pivot_total_income[1])*100

In [76]:
#рассмотрим таблицу с добавленными столбцами
print('Сводная таблица с добавленными столбцами:')
df_pivot_total_income

Сводная таблица с добавленными столбцами:


debt,0,1,% просрочки,% возврата
total_income_group,,,,
"(20666.999, 92140.667]",3291,285,7.969799,92.030201
"(92140.667, 119260.667]",3279,296,8.279720,91.720280
"(119260.667, 142594.0]",3362,329,8.913574,91.086426
"(142594.0, 172357.0]",3291,303,8.430718,91.569282
"(172357.0, 228882.0]",3163,278,8.079047,91.920953
"(228882.0, 2265604.0]",3326,250,6.991051,93.008949


### Вывод

После проведения вычислений видим, что самый низкий процент просрочки кредита, и, соответственно, самый высокий процент возврата кредитов у заёмщиков с заработком от 228 8820 до 2 265 604.0.
Далее в рейтинге просрочки идут люди с заработком от 20 666.999 до 92 140.667.
Третье место по просрочке кредитов у заёмщиков с заработком от 172 357.0 до 228 882.0.
Четвертое место по просрочке кредитов у людей с заработком от 92 140.667 до 119 260.667.
Пятое место у заёмщиков с заработком от 142 594.0 до 172 357.0.
И замыкают рейтинг граждане с заработком от 119 260.667 до 142 594.0, у них самый высокий процент просрочки кредитов и, соответственно, самый низкий процент возврата.

- Как разные цели кредита влияют на его возврат в срок?

In [77]:
#выведем статистику по категориям целей кредитов
print('Cтатистика по категориям целей кредитов:')
print(df['purpose_category'].value_counts()) 

Cтатистика по категориям целей кредитов:
недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2323
Name: purpose_category, dtype: int64


In [78]:
#создадим сводную таблицу для обобщения данных
print('Сводная таблица:')
df_pivot_purpose_category = df.pivot_table(index = 'purpose_category', columns='debt', values='children', aggfunc='count')
df_pivot_purpose_category

Сводная таблица:


debt,0,1
purpose_category,,
автомобиль,3903,403
недвижимость,10029,782
образование,3643,370
свадьба,2137,186


In [79]:
#вычислим % просроченных и возвращённых кредитов для каждой категории
df_pivot_purpose_category['% просрочки'] = df_pivot_purpose_category[1] / (df_pivot_purpose_category[0] + df_pivot_purpose_category[1])*100
df_pivot_purpose_category['% возврата'] = df_pivot_purpose_category[0] / (df_pivot_purpose_category[0] + df_pivot_purpose_category[1])*100

In [80]:
#рассмотрим таблицу с добавленными столбцами
print('Сводная таблица с добавленными столбцами:')
df_pivot_purpose_category

Сводная таблица с добавленными столбцами:


debt,0,1,% просрочки,% возврата
purpose_category,,,,
автомобиль,3903,403,9.359034,90.640966
недвижимость,10029,782,7.233373,92.766627
образование,3643,370,9.220035,90.779965
свадьба,2137,186,8.006888,91.993112


### Вывод

После проведения вычислений видим, что самый низкий процент просрочки кредита, и, соответственно, самый высокий процент возврата кредитов у заёмщиков, которые берут кредит на операции с недвижимостью. Это ожидаемый результат, т.к. если, например,  человек решается взять квартиру в ипотеку и внести немалый первоначальный взнос, то, скорее всего, он будет выплачивать кредит, т.к. в противном случае банк отберёт квартиру, а внесенный первоначальный взнос и внесенные платежи возвращены не будут.
Далее в рейтинге просрочки идут люди, которые берут кредит на свадьбу.
Третье место по просрочке кредитов у заёмщиков, которые берут кредит на образование.
И замыкают рейтинг граждане, которые берут кредит на автомобиль, у них самый высокий процент просрочки кредитов и, соответственно, самый низкий процент возврата. Также ожидаемый результат, т.к. первоначальный взнос по сравнению с квартирой значительно меньше, а в случае неуплаты банк отбирает машину в счёт кредита. На такой случай у граждан есть общественный траспорт. 
Вариант остаться без квартиры всё-таки более пугающий для заёмщика, чем остаться без машины.

### Шаг 4. Общий вывод

В данном проекте перед нами стояла цель исследовать надёжность заёмщиков для кредитного отдела банка.
Необходимо было разобраться, влияют ли такие показатели, как количество детей, семейное положение, уровень дохода клиента, цели кредита на факт погашения кредита в срок.
В качестве входных данных была предоставлена выгрузка со статистикой о платёжеспособности клиентов в формате .csv.

Мы изучили данные, просмотрели значения по каждому столбцу в наборе данных, обработали пропущенные и подозрительные значения:
 - В столбце с информацией о детях 'children' самая многочисленная группа - 'нет детей' (14 090 человек). В столбце были выявлены подозрительные значения -1 и 20. Данные артефакты считали опечаткой при внесении данных. Скорректировали значения 20 на 2, -1 на 1.
 - В столбце с общим трудовым стажем 'days_employed' были выявлены пропущенные, отрицательные, а также чрезвычайно завышенные значения трудового стажа.
  - Отрицательные значения считали опечаткой и перевели данные значения в положительные.
  - По завышенным значениям трудового стажа (максимальное значение - более 1000 лет) считали, что данные были внесены не в днях, а  ошибочно в часах. Все значения в столбце сравнили со средним значением трудового стажа в 35 лет (из интернета), и те, что превышали, поделили на 24 часа, чтобы получить значение в днях. 
  - Максимальный трудовой стаж в наборе данных после корректировки составляет  45 лет.
  - Минимальный трудовой стаж в наборе данных после корректировки составляет  24 дня.
  - Для замены пропущенных значений сгруппировали значения по типу занятости, посмотрели медианное значение трудового стажа для каждой группы и заменили пропущенные значения медианным значением для соответствующей группы. 
 - В столбце с информацией о возрасте клиента 'dob_years' были выявлены младенцы - клиенты с возрастом '0' лет. Данные значения были заменены медианным для данного столбца значением в 42 года. 
  - Минимальный возраст заёмщиков после корректировки: 19 лет.
  - Максимальный возраст заёмщиков: 75 лет.
 - В столбце 'education' максимальное значение для среднего уровня образования (более 13 тыс.). Данные в столбце представлены в разном регистре. Регистр скорректировали при обработке дубликатов.
 - В столбцe 'gender' встречается пол XNA, расшифровку которого найти не удалось. Данное значение не удаляли, т.к. на поставленные вопросы это не влияет.
  - Среди заёмщиков больше представителей женского пола  - 14236 человек. 
 - В столбце с ежемесячным доходом 'total_income' были выявлены пропущенные значения. Для замены пропущенных значений сгруппировали значения по типу занятости (аналогично, как со столбцом 'days_employed'), посмотрели медианное значение трудового стажа для каждой группы и заменили пропущенные значения медианным значением для соответствующей группы. 
  - Максимальное значение дохода заёмщика в наборе данных: 2 265 604
  - Минимальное значение дохода заёмщика в наборе данных: 20 667
 - В остальных столбцах данные отображались корректно и не требовали внесения правок.
 - В столбце 'family_status' наибольшее значение у категории 'женат/замужем' 12339 человек, наименьшее у категории 'вдовец/вдова' - 959 человек.
 - Среди заёмщиков больше всего сотрудников (10014 человек) и меньше всего студентов (1), предпринимателей(1), женщин в декрете (1) и безработных (2).
 - Чаще всего кредит берут на недвижимость (10811 человек), реже всего на свадьбу (2323 человек)
 - Подсчитали средний процент просрочки  -  8%.

После обработки пропущенных и подозрительных значений  убедились, что суммарное количество пропусков для каждого столбца равно нулю.

Далее заменили вещественный тип данных в столбцах 'days_employed', 'total_income' на целочисленный, удалили дубликаты в наборе данных, в том числе с учётом регистра.

Провели лемматизацию целей кредита в столбце 'purpose' при помощи библиотеки pymystem3. Лемматизированнные слова выделили в отдельный столбец и добавили к общему набору данных. 

Провели категоризацию данных о целях кредитов, доходах, детях:
 - Выделили несколько категорий целей - свадьба, недвижимость, автомобиль, образование.
 - Выделили категории наличия детей - нет детей, один ребёнок, двое детей, трое детей, четверо детей, пятеро детей
 - Поделили значения дохода на равные группы при помощи метода qcut(): от 119260.667 до  142594.0, от 142594.0 до 172357.0,        от 228882.0 до 2265604.0, от 20666.999 до 92140.667, от 92140.667 до 119260.667, от 172357.0 до 228882.0.
Столбцы с категориями добавили к набору данных. И приступили к ответам на поставленные вопросы.

Для ответа на вопрос 'Есть ли зависимость между наличием детей и возвратом кредита в срок?' построили сводную таблицу с количеством элементов для каждой категории. Для вычисления % просроченных кредитов создали новый столбец. Для этого поделили значения в столбце с задолжностью на суммарное значение для категории. Для вычисления % возвращённых кредитов также создали новый столбец, для чего поделили значения в столбце без задолжности на суммарное значение для категории.
 - В результате увидели, что зависимость между наличием детей и возвратом кредита в срок есть. Самый низкий процент просрочки кредита, и, соответственно, самый высокий процент возврата кредитов у людей без детей. Далее в рейтинге идут люди, у которых трое детей. Третье место по просрочке кредитов у людей с одним ребёнком. Четвертое место по просрочке кредитов у людей с двумя детьми. И завершают рейтинг люди с четырьмя детьми, у них самый высокий процент просрочки кредитов. Для категории людей, у которых пятеро детей, данные по невозвращённым кредитам отсутствуют, т.е. формально они самые ответственные, но группа оказалась немногочисленная (9 человек), поэтому не стали брать их в расчёт.

Для ответа на остальные вопросы 'Есть ли зависимость между семейным положением и возвратом кредита в срок?', 'Есть ли зависимость между уровнем дохода и возвратом кредита в срок?', 'Как разные цели кредита влияют на его возврат в срок?' провели аналогичные вычисления.
- 'Есть ли зависимость между семейным положением и возвратом кредита в срок?' - После проведения вычислений видим, что зависимость между семейным положением и возвратом кредита в срок есть. Самый низкий процент просрочки кредита, и, соответственно, самый высокий процент возврата кредитов у вдовствующих граждан. Далее в рейтинге идут разведённые заёмщики. Третье место по просрочке кредитов у женатых людей. Четвертое место по просрочке кредитов у тех, кто живёт гражданским браком. И замыкают рейтинг неженатые граждане, у них самый высокий процент просрочки кредитов и, соответственно, самый низкий процент возврата.
- 'Есть ли зависимость между уровнем дохода и возвратом кредита в срок?' - Зависимость между уровнем дохода и возвратом кредита в срок также есть. Самый низкий процент просрочки кредита, и, соответственно, самый высокий процент возврата кредитов у заёмщиков с заработком от 228 8820 до 2 265 604.0. Далее в рейтинге просрочки идут люди с заработком от 20 666.999 до 92 140.667. Третье место по просрочке кредитов у заёмщиков с заработком от 172 357.0 до 228 882.0. Четвертое место по просрочке кредитов у людей с заработком от 92 140.667 до 119 260.667. Пятое место у заёмщиков с заработком от 142 594.0 до 172 357.0. И замыкают рейтинг граждане с заработком от 119 260.667 до 142 594.0, у них самый высокий процент просрочки кредитов и, соответственно, самый низкий процент возврата.
- 'Как разные цели кредита влияют на его возврат в срок?' - Разные цели кредита влияют на его возврат в срок. Самый низкий процент просрочки кредита, и, соответственно, самый высокий процент возврата кредитов у заёмщиков, которые берут кредит на операции с недвижимостью. Далее в рейтинге просрочки идут люди, которые берут кредит на свадьбу. Третье место по просрочке кредитов у заёмщиков, которые берут кредит на образование. И замыкают рейтинг граждане, которые берут кредит на автомобиль, у них самый высокий процент просрочки кредитов и, соответственно, самый низкий процент возврата.

Обобщая всё вышесказанное, можно сказать, что самыми надёжными являются вдовствующие заёмщики без детей с заработком с заработком от 228 8820 до 2 265 604.0, которые берут кредит на недвижимость.

Самыми ненадёжными являются неженатые заёмщики с четырьмя детьми с заработком  от 119 260.667 до 142 594.0, которые берут кредит на автомобиль.